In [1]:
import numpy as np
import pandas as pd

pd.options.display.max_rows = 1000000 

%matplotlib inline
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams["font.family"] = 'NanumBarunGothic'
plt.rcParams["font.size"] = 20
plt.rcParams["figure.figsize"] = (15,8)

mpl.rc('axes', labelsize=12)
mpl.rc('xtick', labelsize=8)
mpl.rc('ytick', labelsize=8)

import seaborn as sns
#corr_set
from itertools import combinations


In [2]:
#check eup-dong list
sample = pd.read_csv('data/2.세종시_표제부.csv')
sample['읍면동'] = sample['대지위치'].str.replace('세종특별자치시 ','')
sample['읍면동'] = sample['읍면동'].str.replace('조치원읍 ','조치원읍')
sample[['1', '2','3']] = sample['읍면동'].str.split(' ', n=2, expand=True)
sample['2'] = sample['2'].str.replace('조치원읍','조치원읍 ')
list_d = list(set(sample['2']))
data_d = pd.DataFrame(list_d,columns={'읍면동'})

#set myn&dong
list_myn = ['소정면', '연서면', '전의면', '금남면','전동면','장군면','연동면','연기면','부강면']
list_dong = ['종촌동', '소담동', '보람동','다정동', '도담동', '합강동', '가람동','반곡동','나성동', 
              '한솔동','산울동','집현동','어진동','고운동', '아름동', '해밀동', '대평동','새롬동']

C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
def make_rentfee_df(a,b,c) :
  df = pd.read_csv(a)
  df['보증금(만원)'] = df['보증금(만원)'].str.replace(',','')
  df = df.astype({'보증금(만원)' : float, '계약년월' : 'str'})
  df['시군구'] = df['시군구'].str.replace('세종특별자치시 ','')
  df.replace({'전세' : 'yr', '월세' : 'mth'},inplace=True)
  
  df_j = make_rentfee_data(df,b,'yr')
  df_m = make_rentfee_data(df,b,'mth')
  
  d = pd.merge(c,df_j, how='outer',on='읍면동')
  f = pd.merge(d,df_m, how='outer',on='읍면동')

  return f

def make_rentfee_data(i, b, c) :

  df = i[i['전월세구분'] == c]
  df[['읍면동', '2']] = df['시군구'].str.split(' ', n=1, expand=True)

  df = df.groupby(['읍면동']).agg({'계약일' : 'count', '보증금(만원)':'mean'})
  x = b+'_'+c+'_cnt'
  y = b+'_'+c+'_price'
  
  df.columns = [x,y]

  return df

In [4]:
ofi = make_rentfee_df('data/10.세종시_오피스텔(전월세)_실거래가.csv','ofi',data_d)
apt = make_rentfee_df('data/7.세종시_아파트(전월세)_실거래가.csv','apt',data_d)
ali = make_rentfee_df('data/8.세종시_연립다세대(전월세)_실거래가.csv','ali',data_d)
# 단독주택 제외 >> 대지단위(명확하게 비교하기가 어려움)
summary = pd.merge(ofi, apt, how='outer', on='읍면동')
summary = pd.merge(summary, ali, how='outer', on='읍면동')

C:\Anaconda\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [5]:
# 상권코드 정리
market = pd.read_csv('data/14.세종시_상권정보.csv')
code = pd.read_csv('data/15.세종시_상권정보_업종코드.csv')
list_main = list(set(code['대분류코드']))
list_middle = list(set(code['중분류코드']))
list_middle = sorted(list_middle)

In [6]:
def make_summary_by_location(a) :
    list_data = list_dong if a == 'dong' else list_myn
    a = summary[summary['읍면동'].isin(list_data)]
    a = a.reset_index(drop=True)

    ex = pd.DataFrame()

    #market divieded
    for i in a['읍면동'] :
        df = market[market['행정동명']==i]
        df = df.groupby('상권업종중분류코드').agg({'행정동명':'count','상권업종중분류명' : 'first','상권업종대분류코드' :'first','상권업종대분류명' : 'first'})
        
        df_T= df.transpose()
        df_T['읍면동'] = i
        df_T['total'] = df['행정동명'].sum()

        list_x = df_T.iloc[0]

        ex = ex.append(pd.Series(list_x), ignore_index=True)
        
    x = pd.merge(a,ex, how='outer', on='읍면동')
    
    return x

In [7]:
# Set the data without nan at market data
dong = make_summary_by_location('dong')
dong_final = dong[dong['total'] != 0]
myn = make_summary_by_location('myn')
myn_final = myn[myn['total'] != 0]

# drop ofi
dong_final = dong_final.sort_index(axis=1)
dong_final = dong_final.drop({'ofi_mth_price','ofi_mth_cnt','ofi_yr_price','ofi_yr_cnt'},axis = 1)

myn_final = myn_final.sort_index(axis=1)
myn_final = myn_final.drop({'ofi_mth_price','ofi_mth_cnt','ofi_yr_price','ofi_yr_cnt'},axis = 1)

In [15]:
check = code[code['대분류코드']=='N']
check['code'] = check['중분류코드'] + check['중분류명']
list_check = list(set(check['code']))
list_check = sorted(list_check)

<ipython-input-15-b87e9d5fc5dc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check['code'] = check['중분류코드'] + check['중분류명']


In [9]:
for i in list_main :
    df_dong01 = dong_final.loc[:,'ali_mth_cnt':]
    df_myn01 = dong_final.loc[:,'ali_mth_cnt':]
    
    list_x = [middle for middle in list_middle if i in middle]
    print(list_x[0],list_x[-1])

    df_dong02 = dong_final.loc[ :,list_x[0]:list_x[-1]]
    df_dong02['읍면동'] = dong_final['읍면동']
    
    df_myn02 = dong_final.loc[ :,list_x[0]:list_x[-1]]
    df_myn02['읍면동'] = dong_final['읍면동']

    corr01 = pd.merge(df_dong01,df_dong02,on='읍면동')
    corr02 = pd.merge(df_myn01,df_myn02,on='읍면동')

    print(i, list(set(code[code['대분류코드']==i]['대분류명'])))

    corr_dong = corr01.corr(method = 'pearson')
    corr_myn = corr02.corr(method = 'pearson')

    corr_dong.to_csv('png/{}'.format(i)+'_dong.csv')
    corr_myn.to_csv('png/{}'.format(i)+'_myn.csv')
    #df_heatmap = sns.heatmap(corr)
    #plt.savefig('png/{}'.format(i)+'.png')

    #plt.clf()


L01 L05
L ['부동산']
O01 O05
O ['숙박']
Q01 Q15
Q ['음식']
D01 D26
D ['소매']
P01 P03
P ['스포츠']
N01 N08
N ['관광/여가/오락']
F01 F20
F ['생활서비스']
R01 R20
R ['학문/교육']


In [33]:
code[code['중분류코드'] == 'Q05']['중분류명']

'닭/오리요리'

In [44]:
def print_corr_set(k, j) :
    df = pd.read_csv('png/{}'.format(k)+'_{}'.format(j)+'.csv')
    df = df.set_index('Unnamed: 0')
    item = list(df.columns)
    list_com = list(combinations(item,2))
    list_com = sorted(list_com)
    for a in list_com :
        (x,y) = a
        name = code[code['중분류코드'] == y]['중분류명'].iloc[-1] if y in list_middle else ""
        corr = df.loc[x,y]
        
        result = corr if corr >= 0.7 else corr if corr <= -0.7 else 2
        print((x,y), name , result) if result != 2 else ""


In [47]:
print_corr_set('D','myn')

('D01', 'D03') 종합소매점 0.7665254858998265
('D01', 'D04') 취미/오락관련소매 0.979199336857139
('D01', 'D08') 사무/문구/컴퓨터 0.7000058788700813
('D01', 'D11') 유아용품 0.8007545287260702
('D01', 'D13') 책/서적/도서 0.7856742013183862
('D01', 'D16') 화장품소매 0.821466256737967
('D01', 'D20') 의약/의료품소매 0.9958705948858224
('D01', 'D23') 자동차/자동차용품 0.9367659069225724
('D02', 'D03') 종합소매점 0.746714879995433
('D02', 'D09') 애견/애완/동물 -0.9428090415820636
('D02', 'D11') 유아용품 0.9707253433941512
('D02', 'D12') 가전제품소매 0.8675276172357089
('D02', 'D13') 책/서적/도서 0.8783100656536796
('D02', 'D23') 자동차/자동차용품 1.0
('D02', 'D26') 시계/귀금속소매 -0.8783100656536799
('D03', 'D04') 취미/오락관련소매 0.7174479188796976
('D03', 'D05') 의복의류 0.7636592748062204
('D03', 'D07') 가정/주방/인테리어 0.9486033778778202
('D03', 'D08') 사무/문구/컴퓨터 0.8261922460299217
('D03', 'D12') 가전제품소매 0.9287013775293232
('D03', 'D13') 책/서적/도서 0.8977193398012133
('D03', 'D16') 화장품소매 0.97378171958144
('D03', 'D18') 사진/광학/정밀기기소매 0.8130408884751156
('D03', 'D20') 의약/의료품소매 0.8418907898006329
('D03

In [49]:
df = pd.read_csv('png/D_Dong.csv')
df = df.set_index('Unnamed: 0')
item = list(df.columns)
com = list(combinations(item,2))
#df.info()
(x,y) = com[1]
print(com[1])
corr = df.loc[x,y]

result = corr if corr > 0.7 else corr if corr > -0.7 else np.nan
result

('ali_mth_cnt', 'ali_yr_cnt')


nan